In [1]:
%cd ..
import xgboost as xgb
import pandas as pd

pd.set_option('display.max_columns', 500)

/run/media/nazif/2F946E411BA61D49/thesis


In [2]:
df = pd.read_csv("results/8_adding_feature_cols.csv")

cols_to_keep = [
    "pred_energy",
    "pred_num_basepairs",
    "pred_seed_basepairs",
    "ta_log10",
    "sps_mean",
    "anchor_a",
    "6mer_seed",
    "match_8",
    "6mer_seed_1_mismatch",
    "compensatory_site",
    "supplementary_site",
    "supplementary_site_2",
    "empty_seed",
    "9_consecutive_match_anywhere",
    "mirna_conservation",

    "seed_8mer",
    "seed_7mer_a1",
    "seed_7mer_m8",
    "seed_compensatory",
    "seed_clash_2",
    "seed_clash_3",
    "seed_clash_4",
    "seed_clash_5",
]

dropped_df = df[cols_to_keep]

In [4]:
# Convert the DataFrame into a DMatrix object
data = xgb.DMatrix(dropped_df)


In [5]:
dropped_df.isna().sum()

pred_energy                     0
pred_num_basepairs              0
pred_seed_basepairs             0
ta_log10                        0
sps_mean                        0
anchor_a                        0
6mer_seed                       0
match_8                         0
6mer_seed_1_mismatch            0
compensatory_site               0
supplementary_site              0
supplementary_site_2            0
empty_seed                      0
9_consecutive_match_anywhere    0
mirna_conservation              0
seed_8mer                       0
seed_7mer_a1                    0
seed_7mer_m8                    0
seed_compensatory               0
seed_clash_2                    0
seed_clash_3                    0
seed_clash_4                    0
seed_clash_5                    0
dtype: int64

In [12]:
model = xgb.Booster()
model.load_model('results/model.xgb')
predictions = model.predict(data)


In [13]:
predictions

array([0.01900677, 0.21765004, 0.89161396, ..., 0.00395883, 0.00254111,
       0.98965704], dtype=float32)

In [ ]:
df["prediction"] = [1 if pred >= 0.5 else 0 for pred in predictions]

df.head(10)

In [ ]:
df[df.id == "1_809687_G_C_MIMAT0000062"][["id", "prediction"]]

In [ ]:
# Step 1: Group by 'id' and check for different 'prediction' values
grouped = df.groupby('id')['prediction'].nunique()

# Step 2: Filter groups where 'prediction' values are different
ids_with_different_predictions = grouped[grouped > 1].index.tolist()

df['is_affected'] = 0  # Initialize the new column with zeros
df.loc[df['id'].isin(ids_with_different_predictions), 'is_affected'] = 1


df[df.is_affected == 1]

In [ ]:
df[df.is_affected == 1][["id", "is_mutated", "prediction"]].sort_values(by=["id", "is_mutated"])

In [ ]:
df[df.is_affected == 1].sort_values(by=["id", "is_mutated"])


In [ ]:

df = df.sort_values(by=["id", "is_mutated"])

# Create a new column to store the changes in prediction
df['change'] = 0

# Initialize variables to keep track of the previous 'id' and 'prediction' values
prev_id = None
prev_prediction = None

# Iterate through the DataFrame row by row
for i, row in df.iterrows():

    if i % 2 == 1:
        # Get the current 'id' and 'prediction' values
        current_id = row['id']
        current_prediction = row['prediction']

        # Check if it's not the first row (i.e., prev_id and prev_prediction are not None)
        if prev_id is not None and prev_prediction is not None:
            # Check if the 'prediction' value decreased from the previous row
            if current_prediction < prev_prediction:
                df.at[i, 'change'] = -1
            # Check if the 'prediction' value increased from the previous row
            elif current_prediction > prev_prediction:
                df.at[i, 'change'] = 1

        # Update the previous row values
        prev_id = current_id
        prev_prediction = current_prediction

# Display the updated DataFrame with the 'change' column


df.head()

In [ ]:
df[df.is_affected == 1]

In [ ]:
df[["id", "prediction"]].to_dict().keys()

In [ ]:
import pandas as pd

# Assuming your DataFrame is called df and the two columns are 'id' and 'value'
result_dict = {}

# Group the DataFrame by 'id' column
grouped = df.groupby('id')

# Iterate over each group and convert it into a dictionary
for group_name, group_df in grouped:
    result_dict[group_name] = group_df['prediction'].tolist()


for key, value in result_dict.items():
    if value == [0,0]:
        result_dict[key] = "unchanged_0"
        
    elif value == [1,1]:
        result_dict[key] = "unchanged_1"
    
    elif value == [0,1]:
        result_dict[key] = "up"
    
    elif value == [1,0]:
        result_dict[key] = "down"

In [ ]:
result_dict

In [ ]:
dff = pd.DataFrame.from_dict(result_dict, orient='index', columns=['prediction'])

dff[dff.prediction == "up"]

In [ ]:
dff.prediction.value_counts()

In [ ]:
dff[dff.prediction == "up"]

In [ ]:
df.head()

In [ ]:
df[df.id == "1_1911011_C_G_MIMAT0000253"]

In [ ]:
df.loc[5424, "mrna_sequence"][22:59]

In [ ]:
'TCACG  TGA  CGA  GGCTGCGCAGAACATGCGTGTGGTAC' "&" "TACCCTGTAGATCCGAATTTGTG"
".((((  ...  ...  ((.(..((((...........((((." "&" "))))))))..).)).....))))"

" ))))  ...  ..   )).)..))))           ))))"
' GUGU  UUA  AG   CCUAGAUGUC           CCAU'

# mutated

In [ ]:
5424
print(df.loc[5424, "mrna_sequence"][22:59])

print(df.loc[16048, "mrna_sequence"][22:59])


In [ ]:
TCACGTGACGAGGCTGCGCAGAACATGCGTGTGGTAC

In [ ]:
'TCACG TGA  GGA GGCTGCG CAGAACATGCGTGTGGTAC' "&" "UACCCUGUAGAUCCGAAUUUGUG"
".(((( ...  ((( ..((((( ............(.((((." "&" ")))))))))).))).....))))"

 ")))) .....)))  .)))))             ) ))))"
 "GUGU UUAAGCCU  AGAUGU             C CCAU"

In [ ]:
")))))))))).))).....))))"[::-1]

In [ ]:
len("))))))))..).)).....))))")

In [ ]:
"TACCCTGTAGATCCGAATTTGTG".replace("T", "U")

In [ ]:
"TCACG     TGACGAGGCTGCG  C   AG  AACATGCGTGTGGTAC"
".((((     ......((.(..(  (   ((  ...........((((."

 "))))           )) )  )..) . ))  .....      ))))"          
 "CACA           AA T  TCGG A TC  TACAG      GGTA"

In [ ]:
def reverse_complement_rna_to_dna(rna_sequence):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    reverse_seq = rna_sequence[::-1]
    return ''.join(complement[base] for base in reverse_seq)

reverse_complement_rna_to_dna("TACCCTGTAGATCCGAATTTGTG")

In [ ]:
df.loc[16048, "mrna_sequence"][22:59]


In [ ]:
")))))))))).))).....))))".count(")")

In [ ]:
len("))))))))))")

In [ ]:
len("))))))))")